<a href="https://colab.research.google.com/github/ToonToonToon/229352-Statical-Learning-For-DataScience-2/blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [2]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [3]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [4]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [6]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes':1, 'no':0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y', 'y_new'], axis=1)
# Split the data BEFORE any transformations


# Print data shape
print(x)
print(y)


       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      no  yes  telephone   may         mon       307         1    999         0  nonexistent
...    ...          ...      ...                  ...     ...     ...  ...  

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
22153,30,admin.,single,university.degree,no,no,no,cellular,aug,thu,180,1,999,0,nonexistent
33513,33,blue-collar,single,basic.4y,no,yes,no,cellular,may,tue,266,2,999,1,failure
29845,33,blue-collar,married,basic.9y,NaN,no,no,cellular,apr,mon,11,3,999,0,nonexistent
12566,50,management,married,university.degree,no,yes,no,cellular,jul,mon,236,1,999,0,nonexistent
6308,45,admin.,married,university.degree,NaN,yes,no,telephone,may,tue,99,6,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40411,45,admin.,married,university.degree,no,no,no,cellular,aug,thu,323,1,999,0,nonexistent
12244,45,admin.,married,high.school,no,no,no,telephone,jul,thu,172,10,999,0,nonexistent
3435,44,technician,married,professional.course,NaN,no,no,telephone,may,thu,284,5,999,0,nonexistent
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,139,1,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [9]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training da

X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])



Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [11]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-2.385645e-16,258.786133,-1.626576e-17,961.839582,0.174396
std,1.000017e+00,260.681957,1.000017e+00,188.463776,0.496297
min,-2.205819e+00,0.000000,-5.649911e-01,0.000000,0.000000
25%,-7.709242e-01,103.000000,-5.649911e-01,999.000000,0.000000
50%,-1.969664e-01,180.000000,-2.060704e-01,999.000000,0.000000
75%,6.639703e-01,320.000000,1.528504e-01,999.000000,0.000000
max,5.542611e+00,4918.000000,1.917565e+01,999.000000,7.000000


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [12]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
40032,university.degree
26529,university.degree
38247,university.degree
11321,basic.9y
40121,high.school
...,...
14731,university.degree
27745,basic.6y
17054,professional.course
662,high.school


- **Ordinal Encoding**

In [13]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [14]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
22153,-0.962243,admin.,single,6.0,no,no,no,cellular,aug,thu,180,-0.564991,999,0,nonexistent
33513,-0.675265,blue-collar,single,1.0,no,yes,no,cellular,may,tue,266,-0.206070,999,1,failure
29845,-0.675265,blue-collar,married,3.0,NaN,no,no,cellular,apr,mon,11,0.152850,999,0,nonexistent
12566,0.950949,management,married,6.0,no,yes,no,cellular,jul,mon,236,-0.564991,999,0,nonexistent
6308,0.472651,admin.,married,6.0,NaN,yes,no,telephone,may,tue,99,1.229613,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40411,0.472651,admin.,married,6.0,no,no,no,cellular,aug,thu,323,-0.564991,999,0,nonexistent
12244,0.472651,admin.,married,4.0,no,no,no,telephone,jul,thu,172,2.665296,999,0,nonexistent
3435,0.376991,technician,married,5.0,NaN,no,no,telephone,may,thu,284,0.870692,999,0,nonexistent
6,1.811886,admin.,married,5.0,no,no,no,telephone,may,mon,139,-0.564991,999,0,nonexistent


Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal X_train 'education' head:")
print(X_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_ord_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(X_train_ord_encoded_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [15]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo]= imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [16]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here

X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot = onehot_encoder.transform(X_test[['job']])

X_train_onehot.shape

(28831, 11)

In [17]:
X_train_onehot

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [18]:
X_train = pd.concat([X_train.reset_index(drop=True),
                               pd.DataFrame(X_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_train

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,0.090012,admin.,divorced,6.0,no,no,no,cellular,jul,mon,101,0.152850,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.675265,unemployed,single,6.0,no,no,no,cellular,nov,thu,153,-0.564991,999,1,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.864142,retired,married,6.0,no,yes,no,cellular,oct,mon,837,-0.206070,999,1,failure,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.866584,blue-collar,married,3.0,no,no,no,telephone,jun,thu,296,-0.564991,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.481503,retired,married,4.0,no,yes,no,cellular,jul,tue,317,-0.564991,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.675265,admin.,single,6.0,no,no,yes,cellular,jul,wed,186,0.511771,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,0.376991,management,married,2.0,no,yes,yes,cellular,mar,thu,218,-0.564991,999,0,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,0.568311,technician,married,5.0,no,yes,no,cellular,jul,fri,13,2.306375,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28829,0.281332,services,married,4.0,no,no,yes,telephone,may,tue,262,-0.564991,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
X_test = pd.concat([X_test.reset_index(drop=True),
                               pd.DataFrame(X_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.962243,admin.,single,6.0,no,no,no,cellular,aug,thu,180,-0.564991,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.675265,blue-collar,single,1.0,no,yes,no,cellular,may,tue,266,-0.206070,999,1,failure,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.675265,blue-collar,married,3.0,NaN,no,no,cellular,apr,mon,11,0.152850,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.950949,management,married,6.0,no,yes,no,cellular,jul,mon,236,-0.564991,999,0,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.472651,admin.,married,6.0,NaN,yes,no,telephone,may,tue,99,1.229613,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,0.472651,admin.,married,6.0,no,no,no,cellular,aug,thu,323,-0.564991,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353,0.472651,admin.,married,4.0,no,no,no,telephone,jul,thu,172,2.665296,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12354,0.376991,technician,married,5.0,NaN,no,no,telephone,may,thu,284,0.870692,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12355,1.811886,admin.,married,5.0,no,no,no,telephone,may,mon,139,-0.564991,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [20]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
# 1. สร้าง Copy
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

In [21]:
# 2. กำหนดรายชื่อคอลัมน์ที่ที่เหลืออยู่
remaining_num_cols = ['pdays', 'previous']
remaining_nom_cols = ['marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [22]:
# 3. จัดการNumericalที่เหลือ
scaler = StandardScaler()
X_train_copy[remaining_num_cols] = scaler.fit_transform(X_train_copy[remaining_num_cols])
X_test_copy[remaining_num_cols] = scaler.transform(X_test_copy[remaining_num_cols])

In [23]:
# 4. จัดการNominalที่เหลือ(Impute + One-Hot + Concat + Drop)
imputer_nom = SimpleImputer(strategy='most_frequent')
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

for col in remaining_nom_cols:
    # 4.1 Imputation
    X_train_copy[[col]] = imputer_nom.fit_transform(X_train_copy[[col]])
    X_test_copy[[col]] = imputer_nom.transform(X_test_copy[[col]])

    # 4.2 One-Hot Encoding
    train_encoded = onehot_encoder.fit_transform(X_train_copy[[col]])
    test_encoded = onehot_encoder.transform(X_test_copy[[col]])
    feature_names = onehot_encoder.get_feature_names_out([col])
    train_encoded_df = pd.DataFrame(train_encoded, columns=feature_names, index=X_train_copy.index)
    test_encoded_df = pd.DataFrame(test_encoded, columns=feature_names, index=X_test_copy.index)

    # 4.3 Concat
    X_train_copy = pd.concat([X_train_copy, train_encoded_df], axis=1)
    X_test_copy = pd.concat([X_test_copy, test_encoded_df], axis=1)

    # 4.4 Drop
    X_train_copy.drop(columns=[col], inplace=True)
    X_test_copy.drop(columns=[col], inplace=True)

In [24]:
# 5. Cleanup
cols_to_clean = ['job', 'duration']
for col in cols_to_clean:
    if col in X_train_copy.columns:
        X_train_copy.drop(columns=[col], inplace=True)
    if col in X_test_copy.columns:
        X_test_copy.drop(columns=[col], inplace=True)

In [25]:
print("Final X_train shape:", X_train_copy.shape)
print("Final X_test shape:", X_test_copy.shape)

Final X_train shape: (28831, 45)
Final X_test shape: (12357, 45)


In [26]:
# 6. Train Model & Evaluate
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_copy, y_train)
y_pred = log_reg.predict(X_test_copy)
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))


--- Classification Report ---
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     10979
           1       0.66      0.19      0.30      1378

    accuracy                           0.90     12357
   macro avg       0.79      0.59      0.62     12357
weighted avg       0.88      0.90      0.87     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


